<p align="center">
  26 de Novembro de 2021
</p>

<br>
<br>

<p align="center">
  <img height="150" src="https://www.ccs.ufscar.br/imagens/ufscar-preto.png">
</p>

<p align="center">
  <img height="150" src="https://site.dc.ufscar.br/static/media/LOGO-DC.295bfc37.svg">
</p>

---

<h1 align="center">Trabalho de Aprendizado de Máquina 2</h1>


<h3 align="center">Profº. Dr. Diego Furtado Silva</h3>

<p align="justify"> Esse trabalho tem o intuito de aplicar cinco datasets multirrótulos diferentes em algoritmos multirrótulos e realizar uma avalicação experimental comparativa da robustez dos algoritmos escolhidos. Assim, Classificação multirrótulo (CMR) pode ser definida como a tarefa de associar múltiplos rótulos de classe para um objeto com base nas características que o descrevem. Atualmente existem muitas aplicações importantes para a tarefa, tais como a categorização de textos (associar documentos texto a tópicos) e a classificação semântica de cenas (associar imagens a conceitos).
 Um dos exemplos é a categorização automática de músicas, onde o objetivo é associar canções a gêneros musicais. Neste problema, temos, por exemplo, que muitas músicas compostas pelo grupo brasileiro Novos Baianos podem serclassificadas como pertencentes aos gêneros “Rock”, “MPB” e “Samba” ao mesmo
tempo. De maneira similar, um grande número de composições do músico Tom Jobim
consiste em uma mistura de dois gêneros: “Jazz” e “Bossa Nova”. Por esta razão, o
problema da categorização de músicas representa um problema de classificação
multirrótulo (CMR – multi-label classification) 
 </p>


<p align="center">
  Lucas Machado Cid (769841) <br>
  Victória De Martini de Souza (759378) <br>
  Vinicius Quaresma da Luz (769836)
</p>

---

O código abaixo importa as dependências necessárias para a posterior execução do código. Antes de executar, certifique-se de ter instalado as dependências declaradas no arquivo `requirements.txt`.

# Declaração de PC bom

In [30]:
import os
pc_bom = os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES')/(1024.**3) > 11

In [31]:
import csv

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, hamming_loss, precision_score, recall_score
from skmultilearn.problem_transform import (BinaryRelevance, ClassifierChain,
                                            LabelPowerset)

As funções abaixo atuam como métodos de conveniência, além de servirem para que o código fique bem modularizado.

In [32]:

def get_data(fileName, possible_labels, total_attributes, limit_instances=0):

  # Abre o arquivo
  file = open('datasets/' + fileName)
  csvreader = csv.reader(file, delimiter =' ')

  data = []

  # Coloca o csv lido em um array
  # O csv tem uma última coluna vazia, por isso a descartamos no processo
  for row in csvreader:
    data.append(row[:-1])

  X = []
  Y = []

  # Pega os dados crus e os divide em atributos e rótulos.
  iteration_limit = len(data) if limit_instances == 0 else limit_instances
  for i in range(iteration_limit):

    # Inicializa o array de atributos para cada elemento
    lx = []
    for j in range(total_attributes):
      lx.append(np.nan)

    # Trata os valores para se encaixar no modelo de dados aceito pelo sklearn
    for j in range(1, len(data[i])):
      attribute = int(data[i][j].split(":")[0]) - 1
      lx[attribute] = float(data[i][j].split(":")[1])

    X.append(lx)

    # Os rótulos obtidos pelo csv estão todos juntos no primeiro atributo, separados por virgula
    # Para cada um dos rótulos possíveis (definido em possible_labels), verifica se o elemento contém ou não cada um dos rótulus
    # e cria novas colunas na tabela com valores 0 ou 1 que indicam se o elemento tem ou não determinado rótulo
    ly = []
    for j in range(len(possible_labels)):
      ly.append(1 if possible_labels[j] in data[i][0].split(",") else 0)
    Y.append(ly)

  # Para valores ausentes, coloca-se a média dos outros elementos para aquele atributo
  imp = SimpleImputer(missing_values=np.nan, strategy='mean')
  X = imp.fit_transform(X)

  return [X, Y]

In [33]:
def get_metrics(classifier, X_train, X_test, Y_train, Y_test):
    classifier.fit(np.array(X_train), np.array(Y_train))

    pred = classifier.predict(X_test)
    
    accuracy = accuracy_score(Y_test, pred)
    hl = hamming_loss(Y_test, pred)
    precision = precision_score(Y_test, pred, average='macro', zero_division=0)
    recall = recall_score(Y_test, pred, average='macro', zero_division=0)

    return {
        'accuracy': accuracy,
        'hamming_loss': hl,
        'precision': precision,
        'recall': recall
    }

In [34]:
class RunResult:
  algorithm: str
  dataset: str
  accuracy_score: float
  hamming_loss: float
  precision: float
  recall: float

  def __init__(self, algorithm, dataset, accuracy_score, hamming_loss, precision, recall):
    self.algorithm = algorithm
    self.dataset = dataset
    self.accuracy_score = accuracy_score
    self.hamming_loss = hamming_loss
    self.precision = precision
    self.recall = recall
  
  def __str__(self):
    return f'algorithm: {self.algorithm}\ndataset: {self.dataset}\naccuracy_score: {self.accuracy_score}\nhamming_loss: {self.hamming_loss}\nprecision: {self.precision}\nrecall: {self.recall}\n\n'
  

results = []

# Algoritmos utilizados

## Binary Relevance
O algoritmo Binary Relevance é o caso mais clássico de transformação de rótulos, visto que o algoritmo transforma os k possíveis rótulos de cada exemplo em k classificadores binários. Cada classificador é treinado para dizer se cada um dos exemplos possui, ou não, um rótulo especifico associonado a ele.

## Encadeamento de Classificadores (CC)
O algoritmo CC tenta resolver o problema de uma possível dependência entres as classes.

## Label Powerset

O algoritmo Label Powerset transforma um problema multirótulo em um problema multiclasse, sendo efetivo quando o problema tem poucos rótulos, ou poucas combinações entre os rótulos, logo que um número alto de rótulos aumenta muito o número de classes possíveis tornando o problema difícil. 

Número de classes vai ser equivalente a 2^n· de rótulos

Os classificadores que serão utilizados com cada um dos 3 algoritmos selecionados são armazenados na lista abaixo para uso posterior.

In [35]:
classifiers = [{"name": "Classifier Chain", "model": ClassifierChain(RandomForestClassifier())}, 
               {"name": "Binary Relevance", "model": BinaryRelevance(RandomForestClassifier())}, 
               {"name": "Label Powerset", "model": LabelPowerset(RandomForestClassifier())}]

In [36]:
def execute(dataset: str, X_train, X_test, Y_train, Y_test):
  for classifier in classifiers:
    result = get_metrics(classifier['model'], X_train, X_test, Y_train, Y_test)

    results.append(RunResult(dataset, classifier['name'], result['accuracy'], result['hamming_loss'], result['precision'], result['recall']))

# Emotions

Este dataset analisa uma série de propriedades musicais e adiciona rótulos à cada exemplo (em que cada exemplo é uma música) de acordo com os sentimentos que a música transmite. Estas fetures contém dados númericos decimais, no qual 8 delas estão relacionados a propriedados como rítmo, e 64 relacionadas com timbre. Já as labels são strings e podem assumir zero ou mais dos seguintes valores: amazed-suprised, happy-pleased, relaxing-calm, quiet-still, sad-lonely e angry-aggresive.


In [37]:
# Dataset emotions
possible_labels = ["amazed-suprised", "happy-pleased", "relaxing-calm", 
                    "quiet-still", "sad-lonely", "angry-aggresive"]
total_attributes = 73

X_test, Y_test = get_data("emotions/emotions_test", possible_labels, total_attributes)
X_train, Y_train = get_data("emotions/emotions_train", possible_labels, total_attributes)

execute('emotions', X_train, X_test, Y_train, Y_test)

# Mediamill
Este dataset está relacionado à detecção automatizada de 101 conceitos semânticos em multimídia. As labels possíveis variam de 1 a 101 e representam diferentes conceitos. Apesar das labels serem númericas, elas indexam um conjunto de conceitos semânticos.

http://jvgemert.github.io/pub/smeulders-search-iciap2007.pdf

In [38]:
# Dataset mediamill
possible_labels = []
total_attributes = 120

for i in range(101):
  possible_labels.append(str(i))

X_train, Y_train = get_data("mediamill/mediamill_train", possible_labels, total_attributes, limit_instances=500)
X_test, Y_test = get_data("mediamill/mediamill_test", possible_labels, total_attributes, limit_instances=100)

execute('mediamill', X_train, X_test, Y_train, Y_test)

## Dataset Yeast

---

Este dataset é um conjunto de dados de leveduras(fermentos) que consiste em uma interação entre proteína-proteína, útil para aqueles que estão começando em alguns projetos de bioinformática de leveduras. O dataset contém dados sobre ontologia genética e dados de sequência de gene/proteína/promotor para cada um dos genes de levedura. Os métodos de detecção de interação levaram à descoberta de milhares de interações entre proteínas, e ajudam fomentar a relevância em conjuntos de dados de grandes escala para a biologia atual.

<br><br>

![levedura.jpg](https://cildata.crbs.ucsd.edu/media/thumbnail_display/50888/50888_thumbnailx512.jpg)
![levedura1.jpg](https://cildata.crbs.ucsd.edu/media/thumbnail_display/50893/50893_thumbnailx512.jpg)


In [39]:
# Dataset Yeast
possible_labels = []
total_attributes = 103

for i in range(15):
  possible_labels.append(str(i))

X_train, Y_train = get_data('yeast/yeast_train', possible_labels, total_attributes)
X_test, Y_test = get_data('yeast/yeast_test', possible_labels, total_attributes)

execute('yeast', X_train, X_test, Y_train, Y_test)

# Scene

Este dataset contém diversas imagens representadas como matrizes de atributos, cada qual relacionada a um ou mais tipos de paisagens, a saber: praia, pôr do sol, folhas de outono, montanha e urbano.

Para cada instância, ocorre um total de 249 features, que compõem uma matriz de atributos que representam a imagem.

In [40]:
possible_labels = ['0', '1', '2', '3', '4', '5']

X_test, Y_test = get_data('scene/scene_test', possible_labels, 294)
X_train, Y_train = get_data('scene/scene_train', possible_labels, 294)

execute('scene', X_train, X_test, Y_train, Y_test)


# TMC 2007

Este dataset contém uma matriz onde cada linha é um documento relativo à relatórios aeroespaciais escritos em texto livre, e cada coluna é a frequência de determinada palavra de interesse.

In [41]:
if pc_bom:
    possible_labels = []

    for i in range(101):
        possible_labels.append(str(i))

    X_test, Y_test = get_data('tmc2007/tmc2007_test', possible_labels, 47152, limit_instances=1000)
    X_train, Y_train = get_data('tmc2007/tmc2007_train', possible_labels, 47152, limit_instances=5000)


    execute('tmc2007', X_train, X_test, Y_train, Y_test)

In [42]:
# Print the results as a table
for result in results:
    print(result)

algorithm: emotions
dataset: Classifier Chain
accuracy_score: 0.30198019801980197
hamming_loss: 0.20297029702970298
precision: 0.7378285838516994
recall: 0.5879570834004224


algorithm: emotions
dataset: Binary Relevance
accuracy_score: 0.28217821782178215
hamming_loss: 0.1971947194719472
precision: 0.768081827592697
recall: 0.5799811641488116


algorithm: emotions
dataset: Label Powerset
accuracy_score: 0.3712871287128713
hamming_loss: 0.19636963696369636
precision: 0.6869592041933196
recall: 0.7193688722201514


algorithm: mediamill
dataset: Classifier Chain
accuracy_score: 0.06
hamming_loss: 0.04
precision: 0.05308022924627958
recall: 0.032411253798190875


algorithm: mediamill
dataset: Binary Relevance
accuracy_score: 0.05
hamming_loss: 0.03881188118811881
precision: 0.04543741391772706
recall: 0.03376080119532691


algorithm: mediamill
dataset: Label Powerset
accuracy_score: 0.02
hamming_loss: 0.055544554455445545
precision: 0.05959786055657334
recall: 0.045584788893635916


algor